In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">

  <td>
<a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/spark/spark_ml.ipynb" target='_blank'>
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/workbench/spark/spark_ml.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

This notebook tutorial runs Apache SparkML jobs with Dataproc and BigQuery to exemplify a common machine learning pipeline use case: data ingestion and cleaning, feature engineering, modeling, and model evaluation.

### Dataset

The [NYC TLC (Taxi and Limousine Commission) Trips](https://console.cloud.google.com/marketplace/product/city-of-new-york/nyc-tlc-trips) (New York taxi and limosine trips data) and [NYC Citi Bike Trips](https://console.cloud.google.com/marketplace/product/city-of-new-york/nyc-citi-bike) (NYC public bicycle sharing system data) datasets are available in [BigQuery Public Datasets](https://cloud.google.com/bigquery/public-data). BigQuery provides free querying of up to 1TB of data each month.

### Objective

This tutorial runs an Apache SparkML job that fetches data from the BigQuery dataset, performs exploratory data analysis, cleans the data, executes feature engineering, trains the model, evaluates the model, outputs results, and saves the model to a Cloud Storage bucket.

This notebook tutorial performs the following steps:

- Sets up a Google Cloud project and Dataproc cluster.
- Creates a Cloud Storage bucket and a BigQuery dataset.
- Configures the spark-bigquery-connector.
- Ingests BigQuery data into a Spark DataFrame.
- Performa Exploratory Data Analysis (EDA).
- Visualizes the data with samples.
- Cleans the data.
- Selects features.
- Trains the model.
- Outputs results.
- Saves the model to a Cloud Storage bucket.
- Deletes the resources created for the tutorial.

### Costs 

This tutorial uses billable components of Google Cloud:

* [Vertex AI](https://cloud.google.com/vertex-ai/pricing)
* [Cloud Storage](https://cloud.google.com/storage/pricing)
* [Dataproc](https://cloud.google.com/dataproc/pricing)
* [BigQuery](https://cloud.google.com/bigquery/pricing)

You can use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Before you begin

### Set up your Google Cloud project:

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you create an account, you receive a $300 credit towards to your compute and storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the BigQuery API, Notebooks API, Vertex AI API, and Dataproc API](https://console.cloud.google.com/flows/enableapi?apiid=bigquery.googleapis.com,notebooks.googleapis.com,aiplatform.googleapis.com,dataproc&_ga=2.209429842.1903825585.1657549521-326108178.1655322249)

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and inserts the value of Python variables prefixed with `$` into the commands.

### Installation

Install the following packages:

In [ ]:
import os

# Google Cloud Notebook requirements:
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

USER_FLAG = ""
# Google Cloud Notebook dependencies to be installed with '--user':
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [ ]:
if os.getenv("IS_TESTING"):
    """
    Since the testing suite doesn't support testing on Dataproc clusters,
    the testing environment is setup to replicate Dataproc via the following steps:
    """
    JAVA_VER = "8u332-b09"
    JAVA_FOLDER = "/tmp/java"
    FILE_NAME = f"openlogic-openjdk-{JAVA_VER}-linux-x64"
    TAR_FILE = f"{JAVA_FOLDER}/{FILE_NAME}.tar.gz"
    DOWNLOAD_LINK = f"https://builds.openlogic.com/downloadJDK/openlogic-openjdk/{JAVA_VER}/openlogic-openjdk-{JAVA_VER}-linux-x64.tar.gz"
    PYSPARK_VER = "3.1.3"

    ! rm -rf $JAVA_FOLDER
    ! mkdir $JAVA_FOLDER
    # Download Open JDK 8. Spark requires Java to execute.
    ! wget -P $JAVA_FOLDER $DOWNLOAD_LINK
    os.environ["JAVA_HOME"] = f"{JAVA_FOLDER}/{FILE_NAME}"
    ! tar -zxf $TAR_FILE -C $JAVA_FOLDER
    ! echo $JAVA_HOME

    # Install PySpark and other packages.
    ! pip install {USER_FLAG} pyspark==$PYSPARK_VER geopandas pyarrow rtree seaborn -q

### Create a Dataproc cluster

The Spark job executed in this notebook tutorial is compute intensive. Since the job can take a significant amount time to complete in a standard notebook environment, this notebook tutorial runs on a Dataproc cluster that is created with the Dataproc Component Gateway and Jupyter component installed on the cluster.

**Existing Dataproc with Jupyter cluster?**: If you have a running Dataproc cluster that has the [Component Gateway and Jupyter component installed on the cluster](https://cloud.google.com/dataproc/docs/concepts/components/jupyter#gcloud-command), you can use it in this tutorial. If you plan to use it, skip this step, and go to `Switch your kernel`.

Set and name and [compute region](https://cloud.google.com/compute/docs/regions-zones#available) for your new cluster. Your `CLUSTER_NAME` must be **unique within your Google Cloud project**. It must start with a lowercase letter, followed by up to 51 lowercase letters, numbers, and hyphens, and cannot end with a hyphen.

In [ ]:
# The testing environment does not use a Dataproc cluster so cluster creation is skipped during testing.
if not os.getenv("IS_TESTING"):
    CLUSTER_NAME = "[your-cluster]"  # @param {type: "string"}
    CLUSTER_REGION = "[your-region]"  # @param {type: "string"}

    if CLUSTER_REGION == "[your-region]":
        CLUSTER_REGION = "us-central1"

    print(f"CLUSTER_NAME: {CLUSTER_NAME}")
    print(f"CLUSTER_REGION: {CLUSTER_REGION}")

In [ ]:
if not os.getenv("IS_TESTING"):
    !gcloud dataproc clusters create $CLUSTER_NAME \
        --region=$CLUSTER_REGION \
        --enable-component-gateway \
        --image-version=2.0 \
        --optional-components=JUPYTER

#### Switch your kernel

Your notebook kernel is listed at the top of the notebook page. Your notebook should run on the Python 3 kernel running on your Dataproc cluster.

Select **Kernel > Change Kernel** from the top menu, then select `Python 3 on CLUSTER_NAME: Dataproc cluster in REGION (Remote)`.

### Set your project ID

Run the following cell to get your project ID.

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

If the previous command has no output, copy your project ID from the project selector in the [Google Cloud console](https://console.cloud.google.com/). Insert the ID in the `[your-project-id]` placeholder, then run the following command:

In [ ]:
if not PROJECT_ID or PROJECT_ID == "":
    PROJECT_ID = "[your-project-id]"  # @param {type: "string"}

In [ ]:
! gcloud config set project $PROJECT_ID -q

### Create a Cloud Storage bucket

The machine learning model created in this tutorial is stored in a Cloud Storage bucket.

#### Region

Before creating a Cloud Storage bucket, redefine the `REGION` variable (when you change the notebook kernel, previously set variables are deleted).

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

# If you do not specify a region, it is set to "us-central1".
if not REGION or REGION == "" or REGION == "[your-region]":
    REGION = "us-central1"

#### UUID

To avoid name collisions you can create a UUID for the current notebook session, then append the UUID to the name of resources such as the Cloud Storage bucket and BigQuery dataset created in this tutorial.

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

Replace the `[your-bucket-name]` placeholder with the name of your Cloud Storage bucket. The name must be unique across all Cloud Storage buckets.

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}/"

# If you do not specify a bucket name, it will be created based on your project ID and the UUID.
if not BUCKET_NAME or BUCKET_NAME == "" or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = f"{PROJECT_ID}{UUID}"
    BUCKET_URI = f"gs://{BUCKET_NAME}/"

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Confirm your access to the Cloud Storage bucket by displaying the bucket's metadata:

In [ ]:
! gsutil ls -L -b $BUCKET_URI

## Tutorial

### Import required libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from geopandas import gpd
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GBTRegressor
# A Spark Session is how you interact with Spark SQL to create Dataframes
from pyspark.sql import SparkSession
# PySpark functions
from pyspark.sql.functions import (col, floor, pandas_udf, to_timestamp, udf,
                                   unix_timestamp)
# These allow us to create a schema for our data
from pyspark.sql.types import BooleanType, DoubleType
from shapely.geometry import Point

**Note**: After importing libraries, if you see `ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/miniconda3/share/proj failed`, you can ignore it. This is due to a bug with a `geopandas` dependency.


### Initialize the SparkSession

To use Apache Spark with BigQuery, you must include the [spark-bigquery-connector](https://github.com/GoogleCloudDataproc/spark-bigquery-connector) when you initialize the SparkSession.

In [ ]:
VER = "0.26.0"
FILE_NAME = f"spark-bigquery-with-dependencies_2.12-{VER}.jar"

if os.getenv("IS_TESTING"):
    # The local testing environment is not configured to read GCS files using Spark. This is a workaround for testing.
    connector = f"https://github.com/GoogleCloudDataproc/spark-bigquery-connector/releases/download/{VER}/{FILE_NAME}"
else:
    connector = f"gs://spark-lib/bigquery/{FILE_NAME}"


# Initialize the SparkSession.
spark = (
    SparkSession.builder.appName("spark-ml-taxi-citibike")
    .config("spark.jars", connector)
    .getOrCreate()
)

### Fetch data from BigQuery

This tutorial uses the 2017 NYC Taxi dataset and the NYC Citi Bike dataset from 2013 to 2018. Since the time range of each dataset differs, match both datasets to the 2017 time period.

In [ ]:
# Load NYC_taxi in Github Activity Public Dataset from BigQuery.
taxi_df = (
    spark.read.format("bigquery")
    .option("table", "bigquery-public-data.new_york_taxi_trips.tlc_green_trips_2017")
    .load()
)


# Load NYC_Citibike in Github Activity Public dataset from BQ.
bike_df = (
    spark.read.format("bigquery")
    .option("table", "bigquery-public-data.new_york_citibike.citibike_trips")
    .load()
)

Use the `to_timestamp()` and `unix_timestamp()` functions to convert `starttime` and `stoptime` to a Unix timestamp type.

A Unix timestamp, which is an integer that represents time as the number of seconds since `January 1st, 1970 at 00:00:00 UTC`, is easy to parse and use across different systems.

In [ ]:
# Represents 2017-01-01 00:00:00
START_2017 = 1483228800
# Represents 2017-12-31 23:59:59
END_2017 = 1514764799

# Convert the type of starttime from a string to a Unix timestamp.
bike_df = bike_df.withColumn(
    "starttime", unix_timestamp(to_timestamp(col("starttime")))
)

# Convert the type of stoptime from a string to a Unix timestamp.
bike_df = bike_df.withColumn("stoptime", unix_timestamp(to_timestamp(col("stoptime"))))

# Filter data so that `bike_df` only contains 2017 trips.
bike_df = bike_df.where(
    (col("starttime") >= START_2017)
    & (col("starttime") <= END_2017)
    & (col("stoptime") >= START_2017)
    & (col("stoptime") <= END_2017)
)

"""
Since notebook runtime scales linearly with the size of data, to limit processing time and compute resource consumption, 
this tutorial uses 20% of the Citi Bike dataset. You can try different values or remove the next line to use all of the data. 
"""
bike_df = bike_df.sample(0.2)

if os.getenv("IS_TESTING"):
    taxi_df = taxi_df.sample(0.0001)
    bike_df = bike_df.sample(0.0001)

### Perform Exploratory Data Analysis (EDA)

As a first step, use Exploratory Data Analysis (EDA) to find patterns, relations, and anomalies in the data using graphs and other visualizations.

Check Taxi dataset data types first.

In [ ]:
taxi_df.printSchema()

Filter out unnecessary columns, and check null counts of the fields. Note that `pickup_location_id` and `dropoff_location_id` were converted from the BigQuery dataset into string format. To use it effectively, reformat these values.

In [ ]:
taxi_df = taxi_df.select(
    "pickup_datetime",
    "dropoff_datetime",
    "trip_distance",
    "fare_amount",
    col("pickup_location_id").cast("int").alias("start_zone_id"),
    col("dropoff_location_id").cast("int").alias("end_zone_id"),
)

taxi_df.describe().show()
taxi_df.show()

From this summary, you are able to know a lot of information.

  * There are over 11 million trip histories for Green Taxi in 2017, which include some abnormal values, such as negative values.
  * Until 2016, exact latitude and longitude were used for pickup and the dropoff locations. This data raised [privacy concerns](https://agkn.wordpress.com/2014/09/15/riding-with-the-stars-passenger-privacy-in-the-nyc-taxicab-dataset/). `pickup_location_id` and `dropoff_location_id` in the dataset corresponded to the [NYC Taxi Zones](https://data.cityofnewyork.us/Transportation/NYC-Taxi-Zones/d3c5-ddgc), roughly based on NYC Department of City Planning’s Neighborhood Tabulation Areas (NTAs), and are meant to indicate approximate neighborhood locations.

Based on numbers shown above, filter out unrealistic values.

  * Remove null values.
  * trip_distance must be in between 0 and 20 (miles) since Manhattan is approximately 13.4 miles long.
  * fare_amount, which does not include taxes and tips, must be in between 0 and 500 (dollars).

In [ ]:
taxi_df = taxi_df.where(
    (col("trip_distance") > 0)
    & (col("trip_distance") < 20)
    & (col("fare_amount") > 0)
    & (col("fare_amount") < 500)
).dropna()

Use the `unix_timestamp()` function to convert `pickup_datetime` and `dropoff_datetime` as a Unix Timestamp type.

After converting `pickup_datetime` and `dropoff_datetime` to `start_time` and `end_time`, you can manipulate these timestamps produce more insteresting columnnar data.

In [ ]:
# Convert the type of datetime from a string to a Unix timestamp.
taxi_df = taxi_df.withColumn("start_time", unix_timestamp(col("pickup_datetime")))
taxi_df = taxi_df.withColumn("end_time", unix_timestamp(col("dropoff_datetime")))

# Calculate if this is a weekday
taxi_df = taxi_df.withColumn(
    "is_weekday",
    ((floor(col("start_time") / 86400) + 4) % 7 > 0)
    & ((floor(col("start_time") / 86400) + 4) % 7 < 6),
)

# Convert start_time to start_time_in_minute.
taxi_df = taxi_df.withColumn(
    "start_time_in_hour", floor((col("start_time") % 86400) / 60 / 60)
)

# Calculate trip_duration.
taxi_df = taxi_df.withColumn("trip_duration", col("end_time") - col("start_time"))

### Perform Feature Engineering

The Taxi dataset contains trips for all NYC boroughs, while the Citi Bike dataset has trips between Manhattan and parts of Brooklyn and the Bronx. To match locations, filter out all boroughs except Manhattan.

To convert the location ID to the borough, download the GeoJSON format of NYC Taxi zones from NYC OpenData.

In [ ]:
# Load the GeoJSON format of NYC Taxi zones. If a URLError is thrown, please re-run this cell.
gdf_zone = gpd.read_file(
    "https://data.cityofnewyork.us/api/geospatial/d3c5-ddgc?method=export&format=GeoJSON"
)

In [ ]:
# Convert the type of "location_id" column.
gdf_zone["location_id"] = gdf_zone["location_id"].astype("long")

# Convert GeoPandas format to a dictionary.
hm = gdf_zone.to_dict("index")

# Create a set that contains location_id if the location is in Manhattan.
manhattan_set = {hm[i]["location_id"] for i in hm if hm[i]["borough"] == "Manhattan"}


@udf(returnType=BooleanType())
def is_in_manhattan(location_id):
    """
    The preprocessing function takes location_id and returns whether the given location_id is in the manhattan_set.
    Args:
        location_id: A number from 0 to 263 that represents NYC Taxi Zone.
    Returns:
        A Boolean value of whether the given location_id is in the manhattan_set.
    """
    return location_id in manhattan_set

In [ ]:
# Trips occur within Manhattan and between different taxi zones.
taxi_df = taxi_df.where(
    (col("start_zone_id") != col("end_zone_id"))
    & (is_in_manhattan(col("start_zone_id")))
    & (is_in_manhattan(col("end_zone_id")))
)

In [ ]:
taxi_df.show()
taxi_df.describe().show()

Prior to visualizing this modified Taxi dataset, do similar feature engineering for the Citi Bike dataset.

In [ ]:
bike_df.printSchema()
bike_df.describe().show()

This dataset reveals interesting information:

  * There are over 2.8 million logs Citi Bike trips in 2017.
  * The current dataset has some abnormal values.
  * `starttime` and `stoptime` are in string format. To use them effectively, reformat them.
  * The Citi Bike dataset does not contain the `location_id` for NYC Taxi Zones; only coordinates data. To convert the coordinates to the NYC Taxi Zones, use Pandas UDF, a Vectorized UDF feature (see [Introducing Pandas UDF for PySpark](https://www.databricks.com/blog/2017/10/30/introducing-vectorized-udfs-for-pyspark.html)).

In [ ]:
@pandas_udf("long")
def preprocess_zone_id(lat: pd.Series, lon: pd.Series) -> pd.Series:
    """
    The preprocessing function takes latitudes and longitudes and returns the corresponding Taxi Zone id.
    Args:
        lat: The latitude of the position.
        lon: The longitude of the position.
    Returns:
        The Taxi Zone id.
    """
    point_var = [Point(xy) for xy in zip(lon, lat)]
    gdf_points = gpd.GeoDataFrame(
        pd.DataFrame({"lat": lat, "lon": lon}), crs="epsg:4326", geometry=point_var
    )
    gdf_joined = gpd.sjoin(gdf_points, gdf_zone, how="left")
    return gdf_joined["location_id"]

In [ ]:
# Rename "tripduration" to "trip_duration".
bike_df = bike_df.withColumnRenamed("tripduration", "trip_duration")

# Check whether the starttime is a weekday or a weekend.
bike_df = bike_df.withColumn(
    "is_weekday",
    ((floor(col("starttime") / 86400) + 4) % 7 > 0)
    & ((floor(col("starttime") / 86400) + 4) % 7 < 6),
)

# Convert starttime to start_time_in_minute
bike_df = bike_df.withColumn(
    "start_time_in_hour", floor((col("starttime") % 86400) / 60 / 60)
)

# Add "start_zone_id" and "end_zone_id" by applying the UDF "preprocess_zone_id".
bike_df = bike_df.withColumn(
    "start_zone_id",
    preprocess_zone_id(
        bike_df["start_station_latitude"], bike_df["start_station_longitude"]
    ),
)
bike_df = bike_df.withColumn(
    "end_zone_id",
    preprocess_zone_id(
        bike_df["end_station_latitude"], bike_df["end_station_longitude"]
    ),
)

# Filter `bike_df` using the following conditions:
# 1) Trips must done between different taxi zones.
# 2) The start time must be earlier than the end time.
# 3) Trips must start and end within Manhattan.
bike_df = bike_df.where(
    (col("start_zone_id") != col("end_zone_id"))
    & (col("starttime") < col("stoptime"))
    & (is_in_manhattan(col("start_zone_id")))
    & (is_in_manhattan(col("end_zone_id")))
)

bike_df = bike_df.select(
    "trip_duration",
    "starttime",
    "stoptime",
    "start_station_latitude",
    "start_station_longitude",
    "end_station_latitude",
    "end_station_longitude",
    "usertype",
    "start_zone_id",
    "end_zone_id",
    "is_weekday",
    "start_time_in_hour",
)

Assuming that Citi Bike users move through Manhattan's streets, which are the perpendicular, the trip distance can be calculated by applying [Manhattan distance](https://en.wikipedia.org/wiki/Taxicab_geometry).

Manhattan distance calculation example: Assume you are at **W 15th St and 9th Ave, where Google NYC is located,** and you are going to **W 33rd St and 5th Ave, where the Empire State building is located**. You move east from 15th Street until you reach 5th Avenue, and then you travel north until you reach W 33rd St. In this case, **W 16th St and 5th Ave** or **W 33rd St and 9th Ave** can be a hinge point. If we set the starting point as **S**, the ending point as **E**, and the hinge point as **H**, the formula of Manhattan distance is `distance(S, H) + distance(H, E)`.

However, additional calculation is needed to obtain the "real" Manhattan distance. First, unlike the Cartesian coordinate system, actual distance in longitude varies with latitude. For example, 1 degree of longitude represents roughly 69 miles at the equator, while it represents approximately 49 miles at 45 degrees north or south latitude. To calculate the exact distance in the real world using latitudes and longitudes, use [Haversine distance](https://en.wikipedia.org/wiki/Haversine_formula).

Also, Manhattan streets are inclined at about 29 degrees from true north, so points should be rotateds S and E by 29 degrees counter-clockwise.

Therefore, the trip distance formula should be `manhattan_distance = haversine_distance(S', H') + haversine_distance(H', E')` where `S'`, `H'`, `E'` are the rotated points.

In [ ]:
@udf(returnType=DoubleType())
def manhattan_dist(lat_start, lon_start, lat_end, lon_end):
    """
    The preprocessing function takes the latitude and longitude of start position and end position,
    and returns the Manhattan distance between them.
    Args:
        lat_start: The latitude of the start position.
        lon_start: The longitude of the start position.
        lat_end: The latitude of the end position.
        lon_end: The longitude of the end position.
    Returns:
        The Manhattan distance between start and end position.
    """
    # Approximate radius of Earth in mile.
    EARTH_RADIUS = 3958.76

    # Approximate inclined degree of the streets in Manhattan.
    THETA = np.radians(-28.904)

    def haversine(pos_start, pos_end):
        """
        The helper function takes start and end position and returns the haversine distance.
        Args:
            pos_start: a latitude and a longitude of start position in np.array form.
            pos_end: a latitude and a longitude of end position in np.array form.
        Returns:
            The Haversine, the spherical distance between `pos_start` and `pos_end` on a sphere, in this case, the earth.
        """
        rad_dist_lat = np.radians(pos_end[0] - pos_start[0])
        rad_dist_lon = np.radians(pos_end[1] - pos_start[1])
        rad_start_lat = np.radians(pos_start[0])
        rad_end_lat = np.radians(pos_end[0])
        dist = 2 * np.arcsin(
            np.sqrt(
                np.sin(rad_dist_lat / 2) ** 2
                + np.cos(rad_start_lat)
                * np.cos(rad_end_lat)
                * np.sin(rad_dist_lon / 2) ** 2
            )
        )
        return EARTH_RADIUS * dist

    def rotate(pos, theta):
        """
        The helper function takes position and the degree theta, and returns rotated position.
        Args:
            pos: a latitude and a longitude of position in np.array form.
            theta: the degree to rotate.
        Returns:
            Rotated position.
        """
        rotate = np.array(
            [[np.cos(theta), np.sin(theta)], [-np.sin(theta), np.cos(theta)]]
        )
        return np.matmul(rotate, pos)

    if not (lat_start and lon_start and lat_end and lon_end):
        return -1

    # Convert positions to np.array format.
    start, end = np.array([lat_start, lon_start]), np.array([lat_end, lon_end])

    # Rotate each positions by 29' using a helper function.
    rotated_start, rotated_end = rotate(start, THETA), rotate(end, THETA)

    # Get rotated hinge point using rotated start and end point.
    rotated_hinge = np.array([rotated_start[0], rotated_end[1]])

    # Re-rotate the hinge point.
    hinge = rotate(rotated_hinge, -THETA)

    # Return the sum of the Haversine distance between start and hinge, hinge and end.
    return float(haversine(start, hinge) + haversine(hinge, end))

In [ ]:
# Calculate the Manhattan distance between "start_station" and "end_station".
bike_df = bike_df.withColumn(
    "trip_distance",
    manhattan_dist(
        col("start_station_latitude"),
        col("start_station_longitude"),
        col("end_station_latitude"),
        col("end_station_longitude"),
    ),
)

In [ ]:
# Filter out the bike_df with the same condition as the taxi_df
bike_df = bike_df.where(
    (col("trip_distance") > 0)
    & (col("trip_distance") < 20)
    & (col("trip_duration") > 0)
    & (col("start_station_latitude") > 40)
    & (col("start_station_latitude") < 41)
    & (col("end_station_latitude") > 40)
    & (col("end_station_latitude") < 41)
    & (col("start_station_longitude") > -75)
    & (col("start_station_longitude") < -73)
    & (col("end_station_longitude") > -75)
    & (col("end_station_longitude") < -73)
)

In [ ]:
bike_df.printSchema()

Check the distributions for the numerical columns. In PySpark, visualizing is expensive since the data is large. 

For example, the NYC Taxi dataset has more than 10M rows. Therefore, approximately 3% of total data (approximately 330k rows) are extracted as a sample.

In [ ]:
if not os.getenv("IS_TESTING"):
    taxi_pd = taxi_df.sample(0.03).toPandas()
else:
    taxi_pd = taxi_df.toPandas()

# Convert "object" type columns to the float type.
taxi_pd["trip_distance"] = taxi_pd["trip_distance"].astype(float)
taxi_pd["fare_amount"] = taxi_pd["fare_amount"].astype(float)
taxi_pd.info()

if not os.getenv("IS_TESTING"):
    bike_pd = bike_df.sample(0.03).toPandas()
else:
    bike_pd = bike_df.toPandas()
bike_pd.info()

Remove extreme data points, and bin the data into a box and histogram.

In [ ]:
TAXI_COLUMNS_TO_SHOW = {"trip_distance", "trip_duration", "fare_amount"}

taxi_pd_filtered = taxi_pd.query(
    "trip_distance > 0 and trip_distance < 20 \
    and trip_duration > 0 and trip_duration < 10000"
)

sns.relplot(
    data=taxi_pd_filtered,
    x="trip_distance",
    y="trip_duration",
    kind="scatter",
)

for column in taxi_pd_filtered.columns:
    if column in TAXI_COLUMNS_TO_SHOW:
        _, ax = plt.subplots(1, 2, figsize=(10, 4))
        taxi_pd_filtered[column].plot(kind="box", ax=ax[0])
        taxi_pd_filtered[column].plot(kind="hist", ax=ax[1])
        plt.title(column)
        plt.figure()
plt.show()

Most columns are skewed right, and the `trip_distance` increases as `trip_duration` increases. Most trips are completed within an hour (3600 sec).

Do a similar analysis for the Citi Bike dataset.

In [ ]:
BIKE_COLUMNS_TO_SHOW = {"trip_distance", "trip_duration"}

bike_pd_filtered = bike_pd.query(
    "trip_distance > 0 and trip_distance < 20 \
    and trip_duration > 0 and trip_duration < 10000"
)

sns.relplot(
    data=bike_pd_filtered,
    x="trip_distance",
    y="trip_duration",
    hue="usertype",
    kind="scatter",
)

The plot for `trip_distance` and `trip_duration` for Citi Bike is different than the same plot for the Taxi dataset.

Many trip durations are longer than expected given the trip distance. The apparent reasons for this behavior are 1) unlike taxis riders, city bike riders are less likely to have a specific trip destination, and 2) occasional Citi Bike users don't park their bicycles at the station. 

To remove outliers, let's assume the average bike speed is at least 2 miles per hour. Let's filter out trips where the `trip_duration` multiplied by speed is less than `trip_distance`, and let's remove the `Customer` user type.

In [ ]:
bike_pd_filtered = bike_pd.query(
    "trip_distance > 0 and trip_distance < 8 \
    and (trip_duration * 0.0005) <= trip_distance \
    and trip_duration > 0 and trip_duration < 10000 \
    and usertype=='Subscriber'"
)

sns.relplot(
    data=bike_pd_filtered,
    x="trip_distance",
    y="trip_duration",
    hue="usertype",
    kind="scatter",
)

for column in bike_pd_filtered.columns:
    if column in BIKE_COLUMNS_TO_SHOW:
        _, ax = plt.subplots(1, 2, figsize=(10, 4))
        bike_pd_filtered[column].plot(kind="box", ax=ax[0])
        bike_pd_filtered[column].plot(kind="hist", ax=ax[1])
        plt.title(column)
        plt.figure()
plt.show()

Based on the visualized data, apply a filter to remove unrealistic values.

In [ ]:
taxi_df = taxi_df.where(
    (col("trip_duration") > 0)
    & (col("trip_duration") < 3600)
    & ((col("trip_duration") * 0.00055) <= col("trip_distance"))
)

bike_df = bike_df.where(
    (col("trip_duration") > 0)
    & (col("trip_duration") < 3600)
    & (col("usertype") == "Subscriber")
    & ((col("trip_duration") * 0.00055) <= col("trip_distance"))
)

### Feature Selection

Not all features in our dataset will be useful. 
Since the purpose of this tutorial is to compare two datasets, we need to use the same features for training. There are some useful features for training the Taxi dataset, such as `fare_amount`, but since the Citi Bike dataset does not have it, it will not be used in this tutorial.

After choose the following columns as features, they must be assembled with `VectorAssembler()`, which is feature transformer that merges multiple columns into a vector column.

In [ ]:
feature_cols = [
    "is_weekday",
    "start_time_in_hour",
    "start_zone_id",
    "end_zone_id",
    "trip_distance",
]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Transform each column into the vector form.
taxi_transformed_data = assembler.transform(taxi_df)
bike_transformed_data = assembler.transform(bike_df)

# Split randomly with training and test data.
(taxi_training_data, taxi_test_data) = taxi_transformed_data.randomSplit([0.95, 0.05])
(bike_training_data, bike_test_data) = bike_transformed_data.randomSplit([0.95, 0.05])

### Training the Model

PySpark provides several Regression Models, including  `LinearRegression`, `GeneralizedLinearRegression`, `DecisionTreeRegressor`, `RandomForestRegressor`, and `GBTRegressor`.
`GBTRegressor`, known as `Gradient Boosted Trees` is a popular regression method using ensembles of decision trees. This algorithm iteratively trains decision trees in order to minimize a loss function.
Although this model is computationally expensive, testing showed that `GBT Regressor` provided the best results.

In [ ]:
# Define GBTRegressor
gbt = GBTRegressor(
    featuresCol="features",
    labelCol="trip_duration",
    predictionCol="pred_trip_duration",
)

# Define evaluator for r2 score.
evaluator_r2 = RegressionEvaluator(
    labelCol=gbt.getLabelCol(), predictionCol=gbt.getPredictionCol(), metricName="r2"
)

# Define evaluator for RMSE error.
evaluator_rmse = RegressionEvaluator(
    labelCol=gbt.getLabelCol(), predictionCol=gbt.getPredictionCol(), metricName="rmse"
)

In [ ]:
# Train a model and get predictions for the Taxi dataset. It takes several minutes.
taxi_gbt_model = gbt.fit(taxi_training_data)
taxi_gbt_predictions = taxi_gbt_model.transform(taxi_test_data)

In [ ]:
# Train a model and get predictions for the Citi Bike dataset. It takes several minutes.
bike_gbt_model = gbt.fit(bike_training_data)
bike_gbt_predictions = bike_gbt_model.transform(bike_test_data)

In [ ]:
# Evaluate the r2 score for the Taxi dataset
taxi_gbt_accuracy_r2 = evaluator_r2.evaluate(taxi_gbt_predictions)
print(f"Taxi Test GBT R2 Accuracy = {taxi_gbt_accuracy_r2}")

# Evaluate the RMSE for the Taxi dataset
taxi_gbt_accuracy_rmse = evaluator_rmse.evaluate(taxi_gbt_predictions)
print(f"Taxi Test GBT RMSE Accuracy = {taxi_gbt_accuracy_rmse}")

In [ ]:
# Evaluate the r2 score for the Citi Bike dataset
bike_gbt_accuracy_r2 = evaluator_r2.evaluate(bike_gbt_predictions)
print(f"Bike Test GBT R2 Accuracy = {bike_gbt_accuracy_r2}")

# Evaluate the RMSE for the Citi Bike dataset
bike_gbt_accuracy_rmse = evaluator_rmse.evaluate(bike_gbt_predictions)
print(f"Bike Test GBT RMSE Accuracy = {bike_gbt_accuracy_rmse}")

### View the result

The trained model for the Taxi dataset shows about 77% for r2 score and 230 root-mean square error (RMSE), while the model for the Citi Bike dataset shows about 71% for r2 score and 250 RMSE. There are some reasons for these relatively lower scores.

* Didn't use the CrossValidator

  * In real machine learning projects, cross-validation is an essential tool that allows us to better utilize data. However, it reruns the training algorithm several times, so it requres a signficant amount of time and computation resources to complete. For more information, see [Cross-validation (statistics)](https://en.wikipedia.org/wiki/Cross-validation_%28statistics%29).

* Didn't use each feature in the Taxi dataset.

  * The Taxi dataset has a `fare_amount` column, which has a strong correlation between the label, `trip_duration`. If that column is included, the model for the Taxi dataset shows 90+% r2 score.
  * The reason why Taxi dataset had limited features is that the purpose of this project is to compare Taxi and the Citi Bike.
  
* Didn't use the exact coordinates for both datasets.

  * Due to privacy concerns, the Taxi dataset no longer provides the precise locations of pickup and dropoff locations, only NYC Taxi zone locations.
  * Taxi Zone is a categorical number that does not represents a geographical zone defined by NYC Taxi.

### Save the model to a Cloud Storage path

In [ ]:
BUCKET_FOLDER = "/tmp/bucket"

# In the testing environment, saving to Cloud Storage bucket routes to the local file system.
if os.getenv("IS_TESTING"):
    ! rm -rf $BUCKET_FOLDER
    ! mkdir $BUCKET_FOLDER
    bike_gbt_model.write().overwrite().save(f"{BUCKET_FOLDER}")
    ! gsutil cp -r $BUCKET_FOLDER $BUCKET_URI
else:
    bike_gbt_model.write().overwrite().save(f"{BUCKET_URI}/")

## Cleaning up

See [Clean up](https://cloud.google.com/vertex-ai/docs/workbench/managed/create-managed-notebooks-instance-console-quickstart#clean-up) to delete your project or the managed notebook created in this tutorial.

### Delete Google Cloud Storage bucket

In [ ]:
DELETE_BUCKET = True

if DELETE_BUCKET:
    ! gsutil rm -r $BUCKET_URI

### Delete Dataproc Cluster

It is not possible to delete the cluster you are currently using unless you switch the kernel to local. To delete it, you need to switch the kernel to local `Python 3` or `PySpark`, set your `CLUSTER_NAME` and `CLUSTER_REGION` manually in the following cell, and execute the `gcloud` command.

See [Deleting a cluster](https://cloud.google.com/dataproc/docs/guides/manage-cluster#console) to delete the Dataproc cluster created in this tutorial.

In [ ]:
CLUSTER_NAME = "[your-cluster-name]"
CLUSTER_REGION = "[your-cluster-region]"

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    ! gcloud dataproc clusters delete $CLUSTER_NAME --region=$CLUSTER_REGION -q